<a href="https://colab.research.google.com/github/GuidoGiacomoMussini/Text_Mining-Lyrics_Analysis/blob/main/5b_MLM_Song_Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
from google.colab import drive
import pandas as pd
import numpy as np
import torch
import random
from tqdm import tqdm as progress_bar
from transformers import BertTokenizer, BertForMaskedLM
from tensorflow.keras.models import load_model

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [30]:
!cp /content/drive/MyDrive/Colab\ Notebooks/Text\ Mining/Utils/MLM_Utils.py /content
import MLM_Utils as MLM

In [31]:
df =  pd.read_hdf(r'/content/drive/MyDrive/Colab Notebooks/Text Mining/Files/lyrics_topic_detection.h5', key='data')

df.head(2)

,Author,Title,Lyrics,verses,processed_lyrics,processed_verses,POS,important_POS,common_words,topic_similarity,topic
0,Fabrizio De André,La Guerra Di Piero,dormi sepolto in un campo di grano non è la ro...,"[dormi sepolto in un campo di grano, non è la...",dormire seppellire campo Grano rosa Tulipano f...,"[dormire seppellire campo Grano, rosa Tulipan...","[[VERB, VERB, ADP, DET, NOUN, ADP, NOUN], [ADV...","[campo, fan, veglia, ombra, sponda, luccio, ca...","[tempo, terra, campo, fan, veglia, ombra, inve...","{'guerra': 0.178, 'morte': 0.173, 'dio': 0.168...",guerra
1,Fabrizio De André,Don Raffaè,io mi chiamo pasquale cafiero e son brigadiero...,"[io mi chiamo pasquale cafiero, e son brigadi...",chere pasquale Cafiero son brigadiero carcere ...,"[chere pasquale Cafiero, son brigadiero carce...","[[PRON, PRON, VERB, ADJ, ADJ], [CCONJ, AUX, NO...","[brigadiero, carcere, cafiero, catenaccio, ser...","[cafè, carcere, ricetta, cumpagno, fortuna, uo...","{'dio': 0.182, 'politica': 0.176, 'morte': 0.1...",dio


In [32]:
#Import BERT
bert = "dbmdz/bert-base-italian-xxl-cased"
tokenizer = BertTokenizer.from_pretrained(bert)

#Upload the models
BERT_Model_v= BertForMaskedLM.from_pretrained(bert, return_dict=True)
model_state_dict_v = torch.load('/content/drive/MyDrive/Colab Notebooks/Text Mining/Models/De_Andre_Model.pth', map_location=torch.device('cpu'))
BERT_Model_v.load_state_dict(model_state_dict_v)

Some weights of the model checkpoint at dbmdz/bert-base-italian-xxl-cased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'cls.seq_relationship.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


<All keys matched successfully>

In [33]:
# Rhyme model
rhyme_model = load_model('/content/drive/MyDrive/Colab Notebooks/Text Mining/Models/Rhyme_detector')

In [34]:
#filter by an author and select a song
df = df[df.Author == 'Giorgio Gaber'].reset_index(drop=True)
verses = df.verses[13]
verses

['da solo',
 " lungo l'autostrada",
 ' alle prime luci del mattino',
 ' a volte spengo anche la radio',
 ' e lascio il mio cuore incollato al finestrino',
 ' lo so del mondo e anche del resto',
 ' lo so che tutto va in rovina',
 ' ma di mattina',
 ' quando la gente dorme',
 ' col suo normale malumore',
 ' mi può bastare un niente',
 ' forse un piccolo bagliore',
 " un'aria già vissuta",
 ' un paesaggio o che ne so',
 ' e sto bene',
 ' io sto bene come uno quando sogna',
 ' non lo so se mi conviene',
 ' ma sto bene che vergogna',
 ' io sto bene',
 ' proprio ora proprio qui',
 ' non è mica colpa mia',
 ' se mi capita così',
 " è come un'illogica allegria",
 ' di cui non so il motivo',
 ' non so che cosa sia',
 ' è come se improvvisamente',
 ' mi fossi preso il diritto',
 ' di vivere il presente',
 ' io sto bene',
 ' questa illogica allegria',
 ' proprio ora proprio qui',
 ' da solo',
 " lungo l'autostrada",
 ' alle prime luci del mattino']

In [35]:
#Simple model
s = []
for verse in verses:
  v = MLM.modify_verse(BERT_Model_v, tokenizer, verse, 100) #100 indicates the number of possible replacement the model will consider
  s.append(v)
  print(v)

da solo
lungo l
alle prime ore del mattino
a volte suona ancora la chitarra
e vedo il mio cuore venire dal pianto
ne va del mondo e poi del mondo
lo sai che si muore in battaglia
ma di dolore
mentre la regina piange
del tuo profondo dolore
ti dovesse basta un niente
solo un lieve dolore
l mai visto
un bacio ma che ne so
e speriamo bene
io vedo solo tranne lei che ride
non lo so se ti servirà
ma sappiamo bene che quest
io so dove
proprio lì proprio lì
non è più colpa mia
se ti risponde così
fu per tanta invidia
di cui non so il nome
non so che cosa sia
è come se rimanda
mi fossi presa il potere
per ingannare il sole
io so dove
una nuova cattedrale
proprio lì proprio lì
da solo
lungo l
alle prime ore del mattino


In [36]:
#complex model
res = MLM.modify_song(verses, BERT_Model_v, tokenizer, 100, rhyme_model, 3)

deriving the rhyme schema
---------------
modify verse by verse the song:

da solo
lungo l
alle prime ore del mattino
a volte suona ancora la batteria
e vedo il mio cuore venire dal pianto
ne va del mondo e poi del mondo
lo sai che si muore in treno
ma di m
mentre la regina parla
del tuo profondo dolore
ti dovesse basta un niente
solo un lieve dolore
l mai sa
un bacio ma che ne so
e speriamo almeno
io vedo solo tranne lei che ride
non lo so se ti servirà
ma sappiamo bene che pu
io so dove
proprio lì proprio così
non è più colpa mio
se ti risponde così
fu per tanta gio
di cui non so il nome
non so che cosa è
è come se rimanda
mi fossi presa il potere
per ingannare il sole
io so dove
una nuova cattedrale
proprio lì proprio così
da solo
lungo l
alle prime ore del mattino


In [37]:
!rm /content/MLM_Utils.py